In [1]:
import urllib.request
import gzip
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# Retrieve Gene Expression Data

In [2]:
# -----------------------------
# Step 0: Download files
# -----------------------------

# GFF file for B. subtilis 168
gff_url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/009/045/GCF_000009045.1_ASM904v1/GCF_000009045.1_ASM904v1_genomic.gff.gz"
urllib.request.urlretrieve(gff_url, "B_subtilis_168.gff.gz")

# Salmon gene counts (from GEO GSE226559)
salmon_url = "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE226nnn/GSE226559/suppl/GSE226559_salmon.merged.gene_counts.tsv.gz"
urllib.request.urlretrieve(salmon_url, "GSE226559_gene_counts.tsv.gz")

('GSE226559_gene_counts.tsv.gz', <http.client.HTTPMessage at 0x7c216447ad90>)

In [3]:
# -----------------------------
# Step 1: Parse GFF gene annotations
# -----------------------------
gff_file = "B_subtilis_168.gff.gz"  # downloaded from NCBI

gff_df = pd.read_csv(
    gff_file,
    sep="\t",
    comment="#",
    header=None,
    compression='gzip'
)
gff_df.columns = ["seqid","source","type","start","end","score","strand","phase","attributes"]

# Keep only 'gene' features
genes_df = gff_df[gff_df["type"] == "gene"].copy()

# Extract gene IDs in format BSU_00010, BSU_00020, etc.
genes_df["gene_id"] = genes_df["attributes"].str.extract(r"ID=gene-(BSU_\d+)")

# Drop rows where gene_id is NaN (nonstandard gene IDs)
genes_df = genes_df.dropna(subset=["gene_id"])

# Remove underscore to match Salmon counts (BSU00010)
genes_df["gene_id"] = genes_df["gene_id"].str.replace("_", "")

# Keep relevant columns
genes_df = genes_df[["gene_id","start","end","strand"]].reset_index(drop=True)

In [4]:
# -----------------------------
# Step 2: Read Salmon gene counts
# -----------------------------
salmon_file = "GSE226559_gene_counts.tsv.gz"

with gzip.open(salmon_file, "rt") as f:
    gene_counts = pd.read_csv(f, sep="\t")

# Set gene_id as index
gene_counts.set_index("gene_id", inplace=True)

# Keep numeric columns only (drop gene_name)
gene_counts_numeric = gene_counts.drop(columns=["gene_name"])

In [5]:
# -----------------------------
# Step 3: Merge gene counts with GFF coordinates
# -----------------------------
expr_annot = gene_counts_numeric.merge(
    genes_df[["gene_id","start","end","strand"]],
    left_index=True,
    right_on="gene_id",
    how="left"
)

In [6]:
# -----------------------------
# Step 4: Compute per-gene expression (sum across samples)
# -----------------------------
expr_annot["expression"] = gene_counts_numeric.sum(axis=1).values

# Create final_df for downstream analysis
final_df = expr_annot[["gene_id","start","end","expression"]].copy()

# -----------------------------
# Step 5: Null distribution from random genes
# -----------------------------
num_random = 100      # number of random genes
p_threshold = 0.05    # significance cutoff

random_genes = final_df.sample(n=num_random, random_state=42)
null_mean = random_genes["expression"].mean()
null_std = random_genes["expression"].std()

# z-Score & p-Value Calculation

In [7]:
# -----------------------------
# Step 6: Log-transform expression
# -----------------------------
# Use log1p to handle zeros
log_expr = np.log1p(final_df["expression"])

# -----------------------------
# Step 7: Null distribution from random genes (on log scale)
# -----------------------------
num_random = 100      # number of random genes
p_threshold = 0.05    # significance cutoff

random_genes = log_expr.sample(n=num_random, random_state=42)
null_mean = random_genes.mean()
null_std = random_genes.std()

# -----------------------------
# Step 8: Compute Z-scores
# -----------------------------
final_df["z_score"] = (log_expr - null_mean) / null_std

# -----------------------------
# Step 9: Convert Z-scores to p-values (one-sided test)
# -----------------------------
final_df["p_value"] = 1 - norm.cdf(final_df["z_score"])

# -----------------------------
# Step 10: Determine significance
# -----------------------------
final_df["significant"] = final_df["p_value"] < p_threshold

# -----------------------------
# Step 11: Inspect results
# -----------------------------
print(final_df.head(10))

      gene_id    start      end  expression   z_score   p_value  significant
0.0  BSU00010    410.0   1750.0    111962.0  0.867594  0.192808        False
1.0  BSU00020   1939.0   3075.0     86537.0  0.786346  0.215832        False
2.0  BSU00030   3206.0   3421.0      2428.0 -0.340693  0.633333        False
3.0  BSU00040   3437.0   4549.0    191219.0  1.036426  0.150002        False
4.0  BSU00050   4567.0   4812.0     12158.0  0.167324  0.433557        False
5.0  BSU00060   4867.0   6783.0    508789.0  1.345102  0.089296        False
6.0  BSU00070   6994.0   9459.0    436139.0  1.296505  0.097401        False
7.0  BSU00080  14847.0  15794.0      6092.0 -0.050611  0.520182        False
8.0  BSU00090  15915.0  17381.0    594765.0  1.394350  0.081606        False
9.0  BSU00100  17534.0  18865.0     88012.0  0.791677  0.214275        False


In [8]:
final_df.describe()

,start,end,expression,z_score,p_value
count,4.174000e+03,4.174000e+03,4.636000e+03,4636.000000,4636.000000
mean,2.134015e+06,2.134900e+06,1.291080e+05,0.129165,0.436776
std,1.201694e+06,1.201689e+06,9.939808e+05,0.922828,0.261635
min,4.100000e+02,1.750000e+03,0.000000e+00,-2.799488,0.003295
25%,1.108899e+06,1.109608e+06,3.107621e+03,-0.262878,0.222007
50%,2.191109e+06,2.191417e+06,1.797300e+04,0.290609,0.385675
75%,3.145938e+06,3.147004e+06,8.098525e+04,0.765432,0.603678
max,4.215255e+06,4.215389e+06,3.938617e+07,2.716919,0.997441


In [9]:
final_df.to_csv('/kaggle/working/final_bacillus.csv')